# 🎬 Federated API Governance — MCP + A2A Discovery Demo

## End-to-end demo: MCP servers, A2A agents, and federated API Center (Prod + Staging)

This notebook deploys a **complete federated API governance architecture** with **production** and **staging** environments — demonstrating how Azure API Management + Azure API Center enable unified discovery across **MCP servers**, **A2A agents**, and **REST APIs**.

### Architecture (all deployed by this notebook)
For each environment (prod + staging):
- **RG 2** — REST→MCP conversion: Weather, Catalog, Order + Calculator APIs with APIM MCP proxies
- **RG 3** — FastMCP containers: Weather, Catalog, Order, Calculator as Container Apps
- **RG 4** — A2A agents: Title, Outline, Summary agents with agent cards + REST APIs
- **RG 1** — **Central API Center** aggregating all APIs from RG 2, 3, 4 (both environments)

### Region Layout
| Resource | Region |
|----------|--------|
| APIM instances (prod + staging) | **westeurope** (StandardV2) |
| Resource groups + infra | **northeurope** |
| Central API Center | **northeurope** |

### Demo story
| Part | Topic | Description |
|------|-------|-------------|
| 1 | **REST→MCP** | Deploy RG 2 — 4 REST APIs converted to MCP servers via APIM policies (prod + staging) |
| 2-3 | **FastMCP Containers** | Deploy RG 3 — 4 containerized MCP servers (prod + staging) |
| 4-5 | **Federation** | Deploy central APIC (RG 1), aggregate from RG 2 + RG 3 (prod → prod env, staging → staging env) |
| 6-7 | **MCP Discovery** | Unified catalog, dynamic invoke, cross-env agent workflow |
| 8-9 | **A2A Agents** | Deploy RG 4, explore A2A agents, aggregate to central (prod + staging) |
| 10 | **Unified Governance** | Full catalog: REST + MCP + A2A, metrics summary across environments |

### Prerequisites
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and signed in
- [Python 3.12+](https://www.python.org/) with the project requirements installed
- An Azure subscription with **Contributor** access

---
## Part 1 — Deploy REST→MCP APIs (RG 2)
---

> Deploy 4 REST APIs with APIM MCP proxy conversion — Weather, Catalog, Order, Calculator.  
> Each API is: REST backend → APIM policy-based MCP conversion → API Center registration.

### 0️⃣ Initialize Variables

In [6]:
import os, sys, json, requests, subprocess
sys.path.insert(1, '../../shared')
import utils

# Ensure correct subscription context
target_sub = "31613fe0-1e9b-4a97-b771-dc48fbaa0fbb"
utils.run(f'az account set --subscription {target_sub}', "", "")

# ═══════════════════════════════════════════════════════════
#  🔢 DEPLOYMENT INDEX — change this to spin up a new demo
#     while the previous one is still deleting
# ═══════════════════════════════════════════════════════════
IDX = 3   # increment for each new demo run (1, 2, 3, …)

# ── APIM SKU — StandardV2 enables built-in analytics reports ──
apim_sku = "Standardv2"

# ── REGION LAYOUT ──
apim_location = "westeurope"     # APIM instances (prod + staging)
rg_location   = "northeurope"    # Resource groups + all other infra
apic_location = "westeurope"     # API Center (not available in northeurope)

# ── ENVIRONMENTS ──
ENVS = ["prod", "staging"]

# Regions that support StandardV2 APIM (as of Feb 2026)
standardv2_regions = [
    "australiaeast", "canadacentral", "centralindia", "centralus",
    "eastasia", "eastus", "eastus2", "francecentral", "germanywestcentral",
    "japaneast", "koreacentral", "northeurope", "norwayeast",
    "southafricanorth", "southcentralus", "southeastasia", "swedencentral",
    "switzerlandnorth", "uaenorth", "uksouth", "westeurope", "westus2", "westus3",
]

if apim_location.lower() not in standardv2_regions:
    utils.print_error(f"⚠️  {apim_sku} is NOT available in '{apim_location}' — falling back to Basicv2")
    apim_sku = "Basicv2"

# ── Per-environment resource configs ──
# Each env dict: rg2 (REST→MCP), rg3 (FastMCP), rg4 (A2A)
envs = {}
for env in ENVS:
    envs[env] = {
        # RG 2: REST→MCP conversion (Weather, Catalog, Order, Calculator)
        "rg2_name": f"rg-lab-mcp-demo-{env}-{IDX}",
        "rg2_apim_name": f"apim-mcp-demo-{env}-{IDX}",
        "rg2_apic_prefix": f"apic-demo-{env}-{IDX}",
        "rg2_deployment_name": f"mcp-demo-initial-{env}",
        "rg2_calc_deployment_name": f"mcp-demo-calculator-{env}",
        # RG 3: FastMCP containers
        "rg3_name": f"rg-lab-mcp-containers-{env}-{IDX}",
        "rg3_apim_name": f"apim-fastmcp-{env}-{IDX}",
        "rg3_apic_prefix": f"apic-fastmcp-{env}-{IDX}",
        "rg3_deployment_name": f"fastmcp-containers-{env}",
        # RG 4: A2A agents (Title, Outline, Summary)
        "rg4_name": f"rg-lab-a2a-demo-{env}-{IDX}",
        "rg4_apim_name": f"apim-a2a-{env}-{IDX}",
        "rg4_apic_prefix": f"apic-a2a-{env}-{IDX}",
        "rg4_deployment_name": f"a2a-initial-{env}",
        "rg4_summary_deployment_name": f"a2a-add-summary-{env}",
    }

# ── RG 1: Central API Center (shared across prod + staging) ──
rg1_name = f"rg-lab-apic-central-{IDX}"
rg1_location = apic_location
rg1_apic_name = f"apic-central-{IDX}"
rg1_deployment_name = "central-apic"

utils.print_ok(f"Variables initialized  (deployment index = {IDX})")
print(f"  APIM SKU:              {apim_sku}")
print(f"  APIM Location:         {apim_location}")
print(f"  APIC Location:         {apic_location}")
print(f"  RG Location:           {rg_location}")
print(f"  Environments:          {', '.join(ENVS)}")
print(f"  RG 1 (Central APIC):   {rg1_name} ({rg1_location})")
for env in ENVS:
    e = envs[env]
    print(f"  ── {env.upper()} ──")
    print(f"    RG 2 (REST→MCP):     {e['rg2_name']}")
    print(f"    RG 3 (FastMCP):      {e['rg3_name']}")
    print(f"    RG 4 (A2A Agents):   {e['rg4_name']}")

⚙️ Running: az account set --subscription 31613fe0-1e9b-4a97-b771-dc48fbaa0fbb 
✅ Variables initialized  (deployment index = 3) ⌚ 11:51:32.215934 
  APIM SKU:              Standardv2
  APIM Location:         westeurope
  APIC Location:         westeurope
  RG Location:           northeurope
  Environments:          prod, staging
  RG 1 (Central APIC):   rg-lab-apic-central-3 (westeurope)
  ── PROD ──
    RG 2 (REST→MCP):     rg-lab-mcp-demo-prod-3
    RG 3 (FastMCP):      rg-lab-mcp-containers-prod-3
    RG 4 (A2A Agents):   rg-lab-a2a-demo-prod-3
  ── STAGING ──
    RG 2 (REST→MCP):     rg-lab-mcp-demo-staging-3
    RG 3 (FastMCP):      rg-lab-mcp-containers-staging-3
    RG 4 (A2A Agents):   rg-lab-a2a-demo-staging-3


### 1️⃣ Deploy RG 2 infrastructure — REST APIs + MCP servers via APIM policies (prod + staging)

Deploys per environment: Log Analytics, App Insights, APIM (westeurope), API Center + 3 REST APIs (Weather, Catalog, Order) each converted to an MCP server via APIM policy.

> ⏱️ First deployment takes ~5-8 minutes per environment (APIM provisioning). Subsequent runs are incremental.

In [4]:
# Deploy RG 2 (REST→MCP) for each environment
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

for env in ENVS:
    e = envs[env]
    print(f"\n{'='*60}")
    print(f"  Deploying RG 2 — {env.upper()}: {e['rg2_name']}")
    print(f"{'='*60}")

    utils.create_resource_group(e["rg2_name"], rg_location)

    bicep_parameters = {
        "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
        "contentVersion": "1.0.0.0",
        "parameters": {
            "apimSku": { "value": apim_sku },
            "apimName": { "value": e["rg2_apim_name"] },
            "apimLocation": { "value": apim_location },
            "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
            "apicLocation": { "value": apic_location },
            "apicServiceNamePrefix": { "value": e["rg2_apic_prefix"] }
        }
    }

    params_file = f'params-demo-initial-{env}.json'
    with open(params_file, 'w') as f:
        f.write(json.dumps(bicep_parameters))

    output = utils.run(
        f"az deployment group create --name {e['rg2_deployment_name']} --resource-group {e['rg2_name']} "
        f"--template-file demo-initial.bicep --parameters {params_file}",
        f"✅ RG 2 [{env}] deployment succeeded",
        f"❌ RG 2 [{env}] deployment failed"
    )


  Deploying RG 2 — PROD: rg-lab-mcp-demo-prod-3
⚙️ Running: az group show --name rg-lab-mcp-demo-prod-3 
👉🏽 Using existing resource group 'rg-lab-mcp-demo-prod-3'
⚙️ Running: az deployment group create --name mcp-demo-initial-prod --resource-group rg-lab-mcp-demo-prod-3 --template-file demo-initial.bicep --parameters params-demo-initial-prod.json 
✅ ✅ RG 2 [prod] deployment succeeded ⌚ 11:45:22.421752 :24s]

  Deploying RG 2 — STAGING: rg-lab-mcp-demo-staging-3
⚙️ Running: az group show --name rg-lab-mcp-demo-staging-3 
👉🏽 Using existing resource group 'rg-lab-mcp-demo-staging-3'
⚙️ Running: az deployment group create --name mcp-demo-initial-staging --resource-group rg-lab-mcp-demo-staging-3 --template-file demo-initial.bicep --parameters params-demo-initial-staging.json 
✅ ✅ RG 2 [staging] deployment succeeded ⌚ 11:49:00.785097 :33s]


### 2️⃣ Retrieve RG 2 outputs + deploy Calculator add-on (prod + staging)

In [7]:
# Retrieve RG 2 deployment outputs + deploy Calculator for each env
for env in ENVS:
    e = envs[env]
    print(f"\n{'='*60}")
    print(f"  Retrieving RG 2 outputs — {env.upper()}: {e['rg2_name']}")
    print(f"{'='*60}")

    output = utils.run(f"az deployment group show --name {e['rg2_deployment_name']} -g {e['rg2_name']}", "", "")

    has_outputs = False
    if output.success and output.json_data:
        outputs = output.json_data.get('properties', {}).get('outputs')
        if outputs:
            has_outputs = True

    if has_outputs:
        e["rg2_apim"]   = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service')
        e["rg2_gateway"] = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway')
        e["rg2_apic"]   = utils.get_deployment_output(output, 'apicServiceName', 'API Center')
        e["rg2_apic_api_env"] = utils.get_deployment_output(output, 'apicApiEnvironmentName', 'APIC API Env')
        e["rg2_apic_mcp_env"] = utils.get_deployment_output(output, 'apicMcpEnvironmentName', 'APIC MCP Env')
        rg2_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
        e["rg2_api_key"] = rg2_subscriptions[0].get("key")
    else:
        utils.print_info("Deployment outputs unavailable — resolving from live resources")
        e["rg2_apim"] = e["rg2_apim_name"]
        o = utils.run(f'az apim show --name {e["rg2_apim_name"]} -g {e["rg2_name"]} --query gatewayUrl -o tsv', "APIM Gateway", "")
        e["rg2_gateway"] = o.text.strip() if o.success else ""
        o = utils.run(f'az apic list -g {e["rg2_name"]} --query "[0].name" -o tsv', "API Center", "")
        e["rg2_apic"] = o.text.strip() if o.success else ""
        e["rg2_apic_api_env"] = "api"
        e["rg2_apic_mcp_env"] = "mcp"
        sub_id = utils.get_current_subscription()
        o = utils.run(
            f'az rest --method POST --url "https://management.azure.com/subscriptions/{sub_id}'
            f'/resourceGroups/{e["rg2_name"]}/providers/Microsoft.ApiManagement/service/{e["rg2_apim_name"]}'
            f'/subscriptions/subscription1/listSecrets?api-version=2022-08-01" --query primaryKey -o tsv',
            "API Key", "")
        e["rg2_api_key"] = o.text.strip() if o.success else ""

    print(f"\n📋 RG 2 [{env}] Resources:")
    print(f"  APIM:     {e['rg2_apim']}")
    print(f"  Gateway:  {e['rg2_gateway']}")
    print(f"  APIC:     {e['rg2_apic']}")
    print(f"  API Key:  ****{e['rg2_api_key'][-4:] if e.get('rg2_api_key') else '????'}")

    # Deploy Calculator add-on
    calc_params = {
        "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
        "contentVersion": "1.0.0.0",
        "parameters": {
            "apimServiceName": { "value": e["rg2_apim"] },
            "apicServiceName": { "value": e["rg2_apic"] },
            "apicApiEnvironmentName": { "value": e["rg2_apic_api_env"] },
            "apicMcpEnvironmentName": { "value": e["rg2_apic_mcp_env"] }
        }
    }

    params_file = f'params-demo-calculator-{env}.json'
    with open(params_file, 'w') as f:
        f.write(json.dumps(calc_params))

    output = utils.run(
        f"az deployment group create --name {e['rg2_calc_deployment_name']} --resource-group {e['rg2_name']} "
        f"--template-file demo-add-calculator.bicep --parameters {params_file}",
        f"✅ Calculator add-on deployed in RG 2 [{env}]",
        f"❌ Calculator deployment failed [{env}]"
    )


  Retrieving RG 2 outputs — PROD: rg-lab-mcp-demo-prod-3
⚙️ Running: az deployment group show --name mcp-demo-initial-prod -g rg-lab-mcp-demo-prod-3 
👉🏽 APIM Service: apim-mcp-demo-prod-3
👉🏽 APIM Gateway: https://apim-mcp-demo-prod-3.azure-api.net
👉🏽 API Center: apic-demo-prod-3-mygqfvu4ynjes
👉🏽 APIC API Env: api
👉🏽 APIC MCP Env: mcp

📋 RG 2 [prod] Resources:
  APIM:     apim-mcp-demo-prod-3
  Gateway:  https://apim-mcp-demo-prod-3.azure-api.net
  APIC:     apic-demo-prod-3-mygqfvu4ynjes
  API Key:  ****a4a9
⚙️ Running: az deployment group create --name mcp-demo-calculator-prod --resource-group rg-lab-mcp-demo-prod-3 --template-file demo-add-calculator.bicep --parameters params-demo-calculator-prod.json 
✅ ✅ Calculator add-on deployed in RG 2 [prod] ⌚ 11:52:28.702528 :44s]

  Retrieving RG 2 outputs — STAGING: rg-lab-mcp-demo-staging-3
⚙️ Running: az deployment group show --name mcp-demo-initial-staging -g rg-lab-mcp-demo-staging-3 
👉🏽 APIM Service: apim-mcp-demo-staging-3
👉🏽 APIM Gat

### 📋 Verify RG 2 APIs — REST + MCP servers in API Center and APIM (prod + staging)

In [8]:
for env in ENVS:
    e = envs[env]
    print(f"\n{'='*60}")
    print(f"  RG 2 [{env.upper()}] — APIs in API Center + APIM")
    print(f"{'='*60}")

    # List APIs in APIC
    output = utils.run(
        f'az apic api list -g {e["rg2_name"]} -n {e["rg2_apic"]} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
        f"Listed APIs in RG 2 [{env}] API Center", "Failed to list APIs")
    if output.success:
        print(output.text)

    # Snapshot
    output = utils.run(f'az apic api list -g {e["rg2_name"]} -n {e["rg2_apic"]} -o json', "", "")
    if output.success and output.json_data:
        e["rg2_apis"] = output.json_data
        rg2_rest = sum(1 for api in e["rg2_apis"] if api.get('kind') == 'rest')
        rg2_mcp  = sum(1 for api in e["rg2_apis"] if api.get('kind') == 'mcp')
        print(f"\n📊 RG 2 [{env}] Total: {len(e['rg2_apis'])} APIs — {rg2_rest} REST, {rg2_mcp} MCP")

    # Show APIM APIs
    subscription_id = utils.get_current_subscription()
    print()
    output = utils.run(
        f'az rest --method GET '
        f'--url "https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{e["rg2_name"]}'
        f'/providers/Microsoft.ApiManagement/service/{e["rg2_apim"]}/apis?api-version=2024-06-01-preview" '
        f'--query "value[].{{Name:name, DisplayName:properties.displayName, Type:properties.type, Path:properties.path}}" -o table',
        f"Listed APIs on APIM '{e['rg2_apim']}'", "Failed")
    if output.success:
        print(output.text)


  RG 2 [PROD] — APIs in API Center + APIM
⚙️ Running: az apic api list -g rg-lab-mcp-demo-prod-3 -n apic-demo-prod-3-mygqfvu4ynjes --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in RG 2 [prod] API Center ⌚ 11:54:10.481262 :6s]
Name              Title                Kind
----------------  -------------------  ------
cloudflare        Cloudflare           mcp
sentry            Sentry               mcp
paypal            Paypal               mcp
plaid             Plaid                mcp
asana             Asana                mcp
intercom          Intercom             mcp
linear            Linear               mcp
atlassian         Atlassian            mcp
square            Square               mcp
swagger-petstore  Swagger Petstore     rest
weather-api       Weather API          rest
catalog-api       Product Catalog API  rest
order-api         Place Order API      rest
weather-mcp       Weather MCP          mcp
order-mcp         Place Order MCP      mcp
catalog

---
## Part 2 — Deploy FastMCP Containers (RG 3) — prod + staging
---

> Deploy 4 containerized FastMCP servers into **separate resource groups per environment** — simulating different team/subscription setups.
> Each server is: Container App → APIM MCP Proxy → API Center registration.

### 3️⃣ Deploy infrastructure + containers via Bicep (prod + staging)

| Layer | Resources |
|-------|-----------|
| **Monitoring** | Log Analytics, Application Insights, MCP Dashboard |
| **API Gateway** | API Management (westeurope) with MCP diagnostics |
| **API Governance** | API Center with MCP environment |
| **Containers** | ACR + ACA Environment + 4 Container Apps |
| **MCP Proxies** | 4× Streamable MCP APIs in APIM |
| **Discoverability** | 4× API Center registrations with VS Code install links |

> ⏱️ First deployment takes ~5-8 minutes per env (APIM provisioning). Subsequent runs are incremental.

In [17]:
# Deploy RG 3 (FastMCP containers) for each environment
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

for env in ENVS:
    e = envs[env]
    print(f"\n{'='*60}")
    print(f"  Deploying RG 3 — {env.upper()}: {e['rg3_name']}")
    print(f"{'='*60}")

    utils.create_resource_group(e["rg3_name"], rg_location)

    bicep_parameters = {
        "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
        "contentVersion": "1.0.0.0",
        "parameters": {
            "apimSku": { "value": apim_sku },
            "apimName": { "value": e["rg3_apim_name"] },
            "apimLocation": { "value": apim_location },
            "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
            "apicLocation": { "value": apic_location },
            "apicServiceNamePrefix": { "value": e["rg3_apic_prefix"] }
        }
    }

    params_file = f'params-demo-containers-{env}.json'
    with open(params_file, 'w') as f:
        f.write(json.dumps(bicep_parameters))

    output = utils.run(
        f"az deployment group create --name {e['rg3_deployment_name']} --resource-group {e['rg3_name']} "
        f"--template-file demo-mcp-containers.bicep --parameters {params_file}",
        f"✅ RG 3 [{env}] deployment succeeded",
        f"❌ RG 3 [{env}] deployment failed"
    )


  Deploying RG 3 — PROD: rg-lab-mcp-containers-prod-3
⚙️ Running: az group show --name rg-lab-mcp-containers-prod-3 
👉🏽 Using existing resource group 'rg-lab-mcp-containers-prod-3'
⚙️ Running: az deployment group create --name fastmcp-containers-prod --resource-group rg-lab-mcp-containers-prod-3 --template-file demo-mcp-containers.bicep --parameters params-demo-containers-prod.json 
✅ ✅ RG 3 [prod] deployment succeeded ⌚ 12:25:07.096991 :46s]

  Deploying RG 3 — STAGING: rg-lab-mcp-containers-staging-3
⚙️ Running: az group show --name rg-lab-mcp-containers-staging-3 
👉🏽 Using existing resource group 'rg-lab-mcp-containers-staging-3'
⚙️ Running: az deployment group create --name fastmcp-containers-staging --resource-group rg-lab-mcp-containers-staging-3 --template-file demo-mcp-containers.bicep --parameters params-demo-containers-staging.json 
✅ ✅ RG 3 [staging] deployment succeeded ⌚ 12:26:54.309053 :43s]


### 4️⃣ Retrieve deployment outputs (prod + staging)

In [18]:
# Retrieve RG 3 deployment outputs for each env
for env in ENVS:
    e = envs[env]
    print(f"\n{'='*60}")
    print(f"  Retrieving RG 3 outputs — {env.upper()}: {e['rg3_name']}")
    print(f"{'='*60}")

    output = utils.run(f"az deployment group show --name {e['rg3_deployment_name']} -g {e['rg3_name']}", "", "")

    has_outputs = False
    if output.success and output.json_data:
        outputs = output.json_data.get('properties', {}).get('outputs')
        if outputs:
            has_outputs = True

    if has_outputs:
        e["rg3_apim_service"]  = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service')
        e["rg3_gateway_url"]   = utils.get_deployment_output(output, 'apimGatewayUrl', 'APIM Gateway')
        e["rg3_apic"]          = utils.get_deployment_output(output, 'apicServiceName', 'API Center')
        e["rg3_acr_name"]      = utils.get_deployment_output(output, 'acrName', 'ACR Name')
        e["rg3_acr_server"]    = utils.get_deployment_output(output, 'acrLoginServer', 'ACR Server')
        e["rg3_weather_url"]   = utils.get_deployment_output(output, 'weatherMcpUrl', 'Weather Container')
        e["rg3_catalog_url"]   = utils.get_deployment_output(output, 'catalogMcpUrl', 'Catalog Container')
        e["rg3_order_url"]     = utils.get_deployment_output(output, 'orderMcpUrl', 'Order Container')
        e["rg3_calculator_url"] = utils.get_deployment_output(output, 'calculatorMcpUrl', 'Calculator Container')
        e["rg3_app_insights"]  = utils.get_deployment_output(output, 'appInsightsName', 'App Insights')
        rg3_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
        e["rg3_api_key"] = rg3_subscriptions[0].get("key")
    else:
        utils.print_info("Deployment outputs unavailable — resolving from live resources")
        e["rg3_apim_service"] = e["rg3_apim_name"]
        o = utils.run(f'az apim show --name {e["rg3_apim_name"]} -g {e["rg3_name"]} --query gatewayUrl -o tsv', "APIM Gateway", "")
        e["rg3_gateway_url"] = o.text.strip() if o.success else ""
        o = utils.run(f'az apic list -g {e["rg3_name"]} --query "[0].name" -o tsv', "API Center", "")
        e["rg3_apic"] = o.text.strip() if o.success else ""
        o = utils.run(f'az acr list -g {e["rg3_name"]} --query "[0].{{n:name,s:loginServer}}" -o json', "ACR", "")
        if o.success and o.json_data:
            e["rg3_acr_name"] = o.json_data["n"]
            e["rg3_acr_server"] = o.json_data["s"]
        for app_name, var_name in [("weather-mcp", "rg3_weather_url"), ("catalog-mcp", "rg3_catalog_url"),
                                    ("order-mcp", "rg3_order_url"), ("calculator-mcp", "rg3_calculator_url")]:
            o = utils.run(f'az containerapp show --name {app_name} -g {e["rg3_name"]} --query "properties.configuration.ingress.fqdn" -o tsv', "", "")
            raw = o.text.strip() if o.success else ""
            # Strip Azure CLI warning lines (e.g. "WARNING: The behavior of this command has been altered...")
            fqdn = "\n".join(line for line in raw.splitlines() if not line.startswith("WARNING:")).strip()
            e[var_name] = f"https://{fqdn}" if fqdn else ""
        o = utils.run(f'az monitor app-insights component show -g {e["rg3_name"]} --query "[0].name" -o tsv', "App Insights", "")
        e["rg3_app_insights"] = o.text.strip() if o.success else ""
        sub_id = utils.get_current_subscription()
        o = utils.run(
            f'az rest --method POST --url "https://management.azure.com/subscriptions/{sub_id}'
            f'/resourceGroups/{e["rg3_name"]}/providers/Microsoft.ApiManagement/service/{e["rg3_apim_name"]}'
            f'/subscriptions/subscription1/listSecrets?api-version=2022-08-01" --query primaryKey -o tsv',
            "API Key", "")
        e["rg3_api_key"] = o.text.strip() if o.success else ""

    print(f"\n📋 RG 3 [{env}] Resources:")
    print(f"  APIM:     {e.get('rg3_apim_service','?')}")
    print(f"  Gateway:  {e.get('rg3_gateway_url','?')}")
    print(f"  APIC:     {e.get('rg3_apic','?')}")
    print(f"  ACR:      {e.get('rg3_acr_name','?')} ({e.get('rg3_acr_server','?')})")
    print(f"  API Key:  ****{e.get('rg3_api_key','')[-4:] if e.get('rg3_api_key') else '????'}")


  Retrieving RG 3 outputs — PROD: rg-lab-mcp-containers-prod-3
⚙️ Running: az deployment group show --name fastmcp-containers-prod -g rg-lab-mcp-containers-prod-3 
👉🏽 APIM Service: apim-fastmcp-prod-3
👉🏽 APIM Gateway: https://apim-fastmcp-prod-3.azure-api.net
👉🏽 API Center: apic-fastmcp-prod-3-m7hknq27w253u
👉🏽 ACR Name: acrm7hknq27w253u
👉🏽 ACR Server: acrm7hknq27w253u.azurecr.io
👉🏽 Weather Container: https://weather-mcp.braveflower-d20cc9bd.northeurope.azurecontainerapps.io
👉🏽 Catalog Container: https://catalog-mcp.braveflower-d20cc9bd.northeurope.azurecontainerapps.io
👉🏽 Order Container: https://order-mcp.braveflower-d20cc9bd.northeurope.azurecontainerapps.io
👉🏽 Calculator Container: https://calculator-mcp.braveflower-d20cc9bd.northeurope.azurecontainerapps.io
👉🏽 App Insights: insights-m7hknq27w253u

📋 RG 3 [prod] Resources:
  APIM:     apim-fastmcp-prod-3
  Gateway:  https://apim-fastmcp-prod-3.azure-api.net
  APIC:     apic-fastmcp-prod-3-m7hknq27w253u
  ACR:      acrm7hknq27w253u 

### 5️⃣ Build and push container images to ACR (prod + staging)

In [19]:
containers = [
    ("weather-mcp",    "src/weather/container"),
    ("catalog-mcp",    "src/product-catalog/container"),
    ("order-mcp",      "src/place-order/container"),
    ("calculator-mcp", "src/calculator/container"),
]

for env in ENVS:
    e = envs[env]
    acr = e.get("rg3_acr_name", "")
    if not acr:
        utils.print_error(f"⚠️  Skipping [{env}] — ACR name not available")
        continue
    print(f"\n🔨 Building containers for [{env.upper()}] → {acr}")
    for image_name, context_dir in containers:
        output = utils.run(
            f"az acr build --registry {acr} --image {image_name}:latest "
            f"--file {context_dir}/Dockerfile {context_dir} --no-logs",
            f"✅ [{env}] Built {image_name}", f"❌ [{env}] Failed to build {image_name}")
        if not output.success:
            break


🔨 Building containers for [PROD] → acrm7hknq27w253u
⚙️ Running: az acr build --registry acrm7hknq27w253u --image weather-mcp:latest --file src/weather/container/Dockerfile src/weather/container --no-logs 
✅ ✅ [prod] Built weather-mcp ⌚ 12:29:16.003155 :11s]
⚙️ Running: az acr build --registry acrm7hknq27w253u --image catalog-mcp:latest --file src/product-catalog/container/Dockerfile src/product-catalog/container --no-logs 
✅ ✅ [prod] Built catalog-mcp ⌚ 12:30:25.998089 :9s]
⚙️ Running: az acr build --registry acrm7hknq27w253u --image order-mcp:latest --file src/place-order/container/Dockerfile src/place-order/container --no-logs 
✅ ✅ [prod] Built order-mcp ⌚ 12:31:35.794222 :9s]
⚙️ Running: az acr build --registry acrm7hknq27w253u --image calculator-mcp:latest --file src/calculator/container/Dockerfile src/calculator/container --no-logs 
✅ ✅ [prod] Built calculator-mcp ⌚ 12:32:47.057147 :11s]

🔨 Building containers for [STAGING] → acr562h4ngubj664
⚙️ Running: az acr build --registry a

### 6️⃣ Update container apps with built images (prod + staging)

In [20]:
for env in ENVS:
    e = envs[env]
    acr_server = e.get("rg3_acr_server", "")
    if not acr_server:
        utils.print_error(f"⚠️  Skipping [{env}] — ACR server not available")
        continue
    print(f"\n🔄 Updating container apps for [{env.upper()}]")
    for image_name, _ in containers:
        output = utils.run(
            f"az containerapp update --name {image_name} --resource-group {e['rg3_name']} "
            f"--image {acr_server}/{image_name}:latest",
            f"✅ [{env}] Updated {image_name}", f"❌ [{env}] Failed to update {image_name}")
        if not output.success:
            break


🔄 Updating container apps for [PROD]
⚙️ Running: az containerapp update --name weather-mcp --resource-group rg-lab-mcp-containers-prod-3 --image acrm7hknq27w253u.azurecr.io/weather-mcp:latest 
✅ ✅ [prod] Updated weather-mcp ⌚ 12:48:16.065813 :31s]
⚙️ Running: az containerapp update --name catalog-mcp --resource-group rg-lab-mcp-containers-prod-3 --image acrm7hknq27w253u.azurecr.io/catalog-mcp:latest 
✅ ✅ [prod] Updated catalog-mcp ⌚ 12:48:49.242757 :33s]
⚙️ Running: az containerapp update --name order-mcp --resource-group rg-lab-mcp-containers-prod-3 --image acrm7hknq27w253u.azurecr.io/order-mcp:latest 
✅ ✅ [prod] Updated order-mcp ⌚ 12:49:20.757305 :31s]
⚙️ Running: az containerapp update --name calculator-mcp --resource-group rg-lab-mcp-containers-prod-3 --image acrm7hknq27w253u.azurecr.io/calculator-mcp:latest 
✅ ✅ [prod] Updated calculator-mcp ⌚ 12:49:52.467317 :31s]

🔄 Updating container apps for [STAGING]
⚙️ Running: az containerapp update --name weather-mcp --resource-group rg-

---
## Part 3 — Test FastMCP Containers (prod + staging)
---

### 🧪 Test all 4 containers — direct and through APIM

MCP Streamable HTTP protocol: POST JSON-RPC → SSE response stream

In [ ]:
def mcp_initialize(endpoint, headers=None):
    """Initialize an MCP session and return the session ID."""
    request = {
        "method": "initialize",
        "params": {
            "protocolVersion": "2025-03-26",
            "capabilities": {},
            "clientInfo": {"name": "demo-client", "version": "1.0"}
        },
        "jsonrpc": "2.0", "id": 0
    }
    h = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if headers:
        h.update(headers)
    try:
        resp = requests.post(endpoint, json=request, headers=h, timeout=30)
        session_id = resp.headers.get("Mcp-Session-Id", "")
        resp.close()
        return session_id
    except Exception:
        return ""

def call_mcp(endpoint, tool_name, arguments, label="", extra_headers=None):
    """Call an MCP tool (with auto-initialize) and return the parsed result."""
    h = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if extra_headers:
        h.update(extra_headers)

    # Initialize to get session
    session_id = mcp_initialize(endpoint, extra_headers)
    if session_id:
        h["Mcp-Session-Id"] = session_id

    request = {
        "method": "tools/call",
        "params": {"name": tool_name, "arguments": arguments},
        "jsonrpc": "2.0", "id": 1
    }
    response = None
    try:
        response = requests.post(endpoint, json=request, stream=True, timeout=30, headers=h)
        if response.status_code == 200:
            for line in response.iter_lines(decode_unicode=True):
                if line and line.startswith("data:"):
                    data = json.loads(line[5:].strip())
                    if "result" in data and "content" in data["result"]:
                        text = data["result"]["content"][0].get("text", "")
                        try:
                            result = json.loads(text)
                        except json.JSONDecodeError:
                            result = text
                        utils.print_ok(f"  {label}: ✅")
                        return result
            utils.print_ok(f"  {label}: ✅ (no data in stream)")
            return None
        else:
            utils.print_error(f"  {label}: HTTP {response.status_code}")
            return None
    except Exception as e:
        utils.print_error(f"  {label}: {e}")
        return None
    finally:
        if response:
            response.close()

def list_mcp_tools(endpoint, label="", extra_headers=None):
    """List available tools on an MCP endpoint."""
    h = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if extra_headers:
        h.update(extra_headers)

    session_id = mcp_initialize(endpoint, extra_headers)
    if session_id:
        h["Mcp-Session-Id"] = session_id

    request = {"method": "tools/list", "params": {}, "jsonrpc": "2.0", "id": 1}
    response = None
    try:
        response = requests.post(endpoint, json=request, stream=True, timeout=30, headers=h)
        if response.status_code == 200:
            for line in response.iter_lines(decode_unicode=True):
                if line and line.startswith("data:"):
                    data = json.loads(line[5:].strip())
                    if "result" in data and "tools" in data["result"]:
                        tools = [t["name"] for t in data["result"]["tools"]]
                        print(f"  {label}: {', '.join(tools)}")
                        return tools
        else:
            utils.print_error(f"  {label}: HTTP {response.status_code}")
        return []
    except Exception as e:
        utils.print_error(f"  {label}: {e}")
        return []
    finally:
        if response:
            response.close()

utils.print_ok("MCP helper functions defined (with session support)")

✅ MCP helper functions defined (with session support) ⌚ 12:56:50.878924 


In [22]:
# Test each container — direct and through APIM — for each environment
for env in ENVS:
    e = envs[env]
    print(f"\n{'='*60}")
    print(f"  🧪 Testing FastMCP Containers — {env.upper()}")
    print(f"{'='*60}")

    print(f"\n🧪 Direct container tests [{env}]")
    print("-" * 50)

    tests = [
        (f"{e.get('rg3_weather_url','')}/weather/mcp",      "get_weather",       {"city": "London"},             "Weather"),
        (f"{e.get('rg3_catalog_url','')}/catalog/mcp",       "search_products",   {"query": "laptop"},            "Catalog"),
        (f"{e.get('rg3_order_url','')}/order/mcp",           "place_order",       {"product_id": "PROD-001", "quantity": 1}, "Order"),
        (f"{e.get('rg3_calculator_url','')}/calculator/mcp", "calculate",         {"operation": "multiply", "a": 7, "b": 8}, "Calculator"),
    ]

    results = {}
    for endpoint, tool, args, label in tests:
        result = call_mcp(endpoint, tool, args, f"{label} (direct)")
        results[f"{label}_direct"] = result is not None
        if result:
            print(f"    → {json.dumps(result, indent=2)[:200]}")

    gw = e.get("rg3_gateway_url", "").rstrip("/")
    api_key = e.get("rg3_api_key", "")
    print(f"\n🧪 APIM proxy tests [{env}] ({gw})")
    print("-" * 50)
    apim_headers = {"api-key": api_key}

    apim_tests = [
        (f"{gw}/weather-mcp",    "get_weather",       {"city": "Paris"},              "Weather"),
        (f"{gw}/catalog-mcp",     "search_products",   {"query": "phone"},             "Catalog"),
        (f"{gw}/order-mcp",       "place_order",       {"product_id": "PROD-002", "quantity": 3}, "Order"),
        (f"{gw}/calculator-mcp",  "calculate",         {"operation": "add", "a": 42, "b": 17}, "Calculator"),
    ]

    for endpoint, tool, args, label in apim_tests:
        result = call_mcp(endpoint, tool, args, f"{label} (APIM)", extra_headers=apim_headers)
        results[f"{label}_apim"] = result is not None
        if result:
            print(f"    → {json.dumps(result, indent=2)[:200]}")

    passed = sum(1 for v in results.values() if v)
    print(f"\n📊 [{env}] Results: {passed}/{len(results)} tests passed")


  🧪 Testing FastMCP Containers — PROD

🧪 Direct container tests [prod]
--------------------------------------------------
✅   Weather (direct): ✅ ⌚ 12:57:00.169278 
    → "{'city': 'London', 'condition': 'Snowy', 'temperature': 29.47, 'humidity': 47.74}"
✅   Catalog (direct): ✅ ⌚ 12:57:01.709770 
    → "[{'id': 'PROD-006', 'name': 'Laptop Backpack', 'category': 'Accessories', 'price': 59.99, 'stock': 300, 'description': 'Water-resistant laptop backpack with USB charging port'}]"
✅   Order (direct): ✅ ⌚ 12:57:03.437504 
    → "{'order_id': 'ORD-50B296C2', 'product_id': 'PROD-001', 'product_name': 'Wireless Mouse', 'quantity': 1, 'unit_price': 29.99, 'total': 29.99, 'status': 'confirmed', 'created_at': '2026-02-11T17:57:03.
✅   Calculator (direct): ✅ ⌚ 12:57:05.019593 
    → "{'operation': 'multiply', 'a': 7.0, 'b': 8.0, 'result': 56.0}"

🧪 APIM proxy tests [prod] (https://apim-fastmcp-prod-3.azure-api.net)
--------------------------------------------------
✅   Weather (APIM): ✅ ⌚ 12:57

### 7️⃣ Verify RG 3's local API Center (prod + staging)

Each container MCP server was auto-registered in RG 3's own API Center during Bicep deployment.

In [23]:
for env in ENVS:
    e = envs[env]
    apic = e.get("rg3_apic", "")
    rg = e["rg3_name"]
    print(f"\n{'='*50}")
    print(f"  RG 3 [{env.upper()}] API Center: {apic}")
    print(f"{'='*50}")

    output = utils.run(
        f'az apic api list -g {rg} -n {apic} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
        f"Listed APIs in RG 3 [{env}] API Center", "Failed")
    if output.success:
        print(output.text)

    output = utils.run(f'az apic api list -g {rg} -n {apic} -o json', "", "")
    if output.success and output.json_data:
        e["rg3_apis"] = output.json_data
        rg3_mcp = sum(1 for api in e["rg3_apis"] if api.get('kind') == 'mcp')
        print(f"\n📊 RG 3 [{env}] Total: {len(e['rg3_apis'])} APIs — {rg3_mcp} MCP Servers")


  RG 3 [PROD] API Center: apic-fastmcp-prod-3-m7hknq27w253u
⚙️ Running: az apic api list -g rg-lab-mcp-containers-prod-3 -n apic-fastmcp-prod-3-m7hknq27w253u --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in RG 3 [prod] API Center ⌚ 12:57:39.978072 :3s]
Name              Title                Kind
----------------  -------------------  ------
asana             Asana                mcp
atlassian         Atlassian            mcp
sentry            Sentry               mcp
cloudflare        Cloudflare           mcp
plaid             Plaid                mcp
paypal            Paypal               mcp
square            Square               mcp
linear            Linear               mcp
intercom          Intercom             mcp
swagger-petstore  Swagger Petstore     rest
weather-mcp       Weather MCP          mcp
order-mcp         Order Service MCP    mcp
calculator-mcp    Calculator MCP       mcp
catalog-mcp       Product Catalog MCP  mcp

⚙️ Running: az apic api l

---
## Part 4 — Deploy Central API Center (RG 1) with prod + staging environments
---

> 💡 **This is the key architecture moment.**  
> The central API Center acts as the **single pane of glass** for all APIs and MCP servers across the organization — with separate environments for **production** and **staging** — regardless of which resource group, subscription, or APIM instance hosts them.

In [24]:
# Create RG 1 and deploy central API Center with prod + staging environments
utils.create_resource_group(rg1_name, rg1_location)

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apicServiceName": { "value": rg1_apic_name },
        "location": { "value": rg1_location }
    }
}

with open('params-central-apic.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {rg1_deployment_name} --resource-group {rg1_name} "
    f"--template-file demo-central-apic.bicep --parameters params-central-apic.json",
    f"✅ Central API Center deployed in {rg1_location} (with prod + staging environments)",
    f"❌ Central API Center deployment failed"
)

if output.success and output.json_data:
    rg1_apic_actual = utils.get_deployment_output(output, 'name', 'Central APIC Name')
    utils.print_info(f"Central APIC: {rg1_apic_actual}")
    print(f"  Environments: api-prod, api-staging, mcp-prod, mcp-staging, a2a-prod, a2a-staging")

⚙️ Running: az group show --name rg-lab-apic-central-3 
👉🏽 Resource group rg-lab-apic-central-3 does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name rg-lab-apic-central-3 --location westeurope --tags source=ai-gateway 
✅ Resource group 'rg-lab-apic-central-3' created ⌚ 12:58:42.216649 :16s]
⚙️ Running: az deployment group create --name central-apic --resource-group rg-lab-apic-central-3 --template-file demo-central-apic.bicep --parameters params-central-apic.json 
✅ ✅ Central API Center deployed in westeurope (with prod + staging environments) ⌚ 12:59:38.068628 :55s]


---
## Part 5 — Aggregate APIs into Central Catalog (prod → prod, staging → staging)
---

> Pull APIs from **all distributed API Centers** into the central catalog.  
> Each API is routed to the correct **prod** or **staging** environment in the central APIC.  
> This simulates a platform team aggregating APIs from multiple product teams.

In [25]:
import subprocess

def get_apic_apis(resource_group, apic_name):
    """Fetch all APIs from an API Center instance."""
    output = utils.run(f'az apic api list -g {resource_group} -n {apic_name} -o json', "", "")
    if output.success and output.json_data:
        return output.json_data
    return []

def get_apic_deployments(resource_group, apic_name, api_name):
    """Fetch deployments for an API to get the runtimeUri."""
    output = utils.run(
        f'az apic api deployment list -g {resource_group} -n {apic_name} --api-id {api_name} -o json', "", "")
    if output.success and output.json_data:
        return output.json_data
    return []

def create_api_via_arm(api_name, kind, title, description):
    """Create an API in central APIC using ARM REST API (supports mcp/a2a kinds)."""
    url = (f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg1_name}"
           f"/providers/Microsoft.ApiCenter/services/{rg1_apic_name}/workspaces/default/apis/{api_name}"
           f"?api-version=2024-06-01-preview")
    body = json.dumps({"properties": {"title": title, "kind": kind, "description": description[:200]}})
    body_file = f"_temp_api_{api_name}.json"
    with open(body_file, 'w') as f:
        f.write(body)
    output = utils.run(f'az rest --method PUT --url "{url}" --body @{body_file}', "", "")
    try:
        os.remove(body_file)
    except:
        pass
    return output.success

def create_deployment_via_subprocess(api_name, deployment_id, title, env_name, runtime_uris):
    """Create an API deployment using subprocess to handle complex JSON escaping."""
    server_json = json.dumps({"runtimeUri": runtime_uris})
    args = [
        "az", "apic", "api", "deployment", "create",
        "-g", rg1_name, "-n", rg1_apic_name,
        "--api-id", api_name,
        "--deployment-id", deployment_id,
        "--title", title,
        "--environment-id", f"/workspaces/default/environments/{env_name}",
        "--definition-id", f"/workspaces/default/apis/{api_name}/versions/1-0-0/definitions/default",
        "--server", server_json
    ]
    result = subprocess.run(args, capture_output=True, text=True, shell=True)
    return result.returncode == 0

def register_api_in_central(api, source_rg, source_apic, source_label, stage):
    """Register a single API from a distributed APIC into the central APIC.
    
    Routes to the correct prod/staging environment:
      kind=rest  + stage=prod → api-prod
      kind=mcp   + stage=staging → mcp-staging
      kind=a2a   + stage=prod → a2a-prod   etc.
    """
    api_name = api['name']
    kind = api.get('kind', 'rest')
    title = api.get('title', api_name)
    description = api.get('description', '') or api.get('summary', '') or f'{title} from {source_label}'

    # Map kind → base env name, then append stage
    env_base_map = {'mcp': 'mcp', 'a2a': 'a2a', 'rest': 'api'}
    env_name = f"{env_base_map.get(kind, 'api')}-{stage}"  # e.g. "api-prod", "mcp-staging"

    # Use a stage-prefixed API name to avoid collisions between prod and staging
    central_api_name = f"{api_name}-{stage}"

    # CLI --type only supports rest/graphql/grpc/soap. Use ARM REST API for mcp/a2a.
    cli_type_map = {'rest': 'rest', 'graphql': 'graphql', 'grpc': 'grpc', 'soap': 'soap'}
    if kind in cli_type_map:
        cmd = (f'az apic api create -g {rg1_name} -n {rg1_apic_name} --api-id {central_api_name} '
               f'--title "{title} ({stage})" --type {cli_type_map[kind]} '
               f'--description "{description[:200]}" ')
        utils.run(cmd, "", "")
    else:
        create_api_via_arm(central_api_name, kind, f"{title} ({stage})", description)

    # Add version
    utils.run(f'az apic api version create -g {rg1_name} -n {rg1_apic_name} --api-id {central_api_name} '
              f'--version-id 1-0-0 --title "1.0.0" --lifecycle-stage {"production" if stage == "prod" else "development"}', "", "")

    # Add definition
    utils.run(f'az apic api definition create -g {rg1_name} -n {rg1_apic_name} --api-id {central_api_name} '
              f'--version-id 1-0-0 --definition-id default --title "Default"', "", "")

    # Get runtimeUri from source deployments
    deployments = get_apic_deployments(source_rg, source_apic, api_name)
    runtime_uris = []
    for dep in deployments:
        uris = dep.get('server', {}).get('runtimeUri', [])
        runtime_uris.extend(uris)

    # Register deployment with subprocess to handle JSON escaping
    if runtime_uris:
        deploy_id = f"{central_api_name}-from-{source_label}"
        deploy_title = f"{title} ({stage} — {source_label})"
        create_deployment_via_subprocess(central_api_name, deploy_id, deploy_title, env_name, runtime_uris)

    return runtime_uris

utils.print_ok("Aggregation functions defined (supports REST, MCP, A2A with prod/staging routing)")

✅ Aggregation functions defined (supports REST, MCP, A2A with prod/staging routing) ⌚ 12:59:52.271198 


### 8️⃣ Sync APIs from RG 2 (mixed REST→MCP) → Central APIC (prod → api-prod/mcp-prod, staging → api-staging/mcp-staging)

In [26]:
# Aggregate from RG 2 (mixed REST→MCP APIs) — prod + staging
rg2_synced = {}
for env in ENVS:
    e = envs[env]
    apis = get_apic_apis(e["rg2_name"], e["rg2_apic"])
    e["rg2_apis"] = apis
    print(f"\n📥 Syncing {len(apis)} APIs from RG 2 [{env}] ({e['rg2_apic']}) → Central APIC ({env} envs)")
    print("=" * 70)

    synced = 0
    for api in apis:
        uris = register_api_in_central(api, e["rg2_name"], e["rg2_apic"], f"rg2-{env}", env)
        uri_str = uris[0] if uris else "—"
        print(f"  ✅ {api['name']:25} ({api.get('kind','?'):4}) → {env}  {uri_str}")
        synced += 1

    rg2_synced[env] = synced
    print(f"\n📊 Synced {synced} APIs from RG 2 [{env}]")

⚙️ Running: az apic api list -g rg-lab-mcp-demo-prod-3 -n apic-demo-prod-3-mygqfvu4ynjes -o json 

📥 Syncing 18 APIs from RG 2 [prod] (apic-demo-prod-3-mygqfvu4ynjes) → Central APIC (prod envs)
⚙️ Running: az rest --method PUT --url "https://management.azure.com/subscriptions/31613fe0-1e9b-4a97-b771-dc48fbaa0fbb/resourceGroups/rg-lab-apic-central-3/providers/Microsoft.ApiCenter/services/apic-central-3/workspaces/default/apis/cloudflare-prod?api-version=2024-06-01-preview" --body @_temp_api_cloudflare-prod.json 
⚙️ Running: az apic api version create -g rg-lab-apic-central-3 -n apic-central-3 --api-id cloudflare-prod --version-id 1-0-0 --title "1.0.0" --lifecycle-stage production 
⚙️ Running: az apic api definition create -g rg-lab-apic-central-3 -n apic-central-3 --api-id cloudflare-prod --version-id 1-0-0 --definition-id default --title "Default" 
⚙️ Running: az apic api deployment list -g rg-lab-mcp-demo-prod-3 -n apic-demo-prod-3-mygqfvu4ynjes --api-id cloudflare -o json 
  ✅ cloudf

### 9️⃣ Sync APIs from RG 3 (FastMCP containers) → Central APIC (prod → mcp-prod, staging → mcp-staging)

In [27]:
# Aggregate from RG 3 (FastMCP containers) — prod + staging
rg3_synced = {}
for env in ENVS:
    e = envs[env]
    apis = get_apic_apis(e["rg3_name"], e.get("rg3_apic", ""))
    e["rg3_apis"] = apis
    print(f"\n📥 Syncing {len(apis)} APIs from RG 3 [{env}] ({e.get('rg3_apic','')}) → Central APIC ({env} envs)")
    print("=" * 70)

    synced = 0
    for api in apis:
        uris = register_api_in_central(api, e["rg3_name"], e.get("rg3_apic", ""), f"rg3-{env}", env)
        uri_str = uris[0] if uris else "—"
        print(f"  ✅ {api['name']:25} ({api.get('kind','?'):4}) → {env}  {uri_str}")
        synced += 1

    rg3_synced[env] = synced
    print(f"\n📊 Synced {synced} APIs from RG 3 [{env}]")

⚙️ Running: az apic api list -g rg-lab-mcp-containers-prod-3 -n apic-fastmcp-prod-3-m7hknq27w253u -o json 

📥 Syncing 14 APIs from RG 3 [prod] (apic-fastmcp-prod-3-m7hknq27w253u) → Central APIC (prod envs)
⚙️ Running: az rest --method PUT --url "https://management.azure.com/subscriptions/31613fe0-1e9b-4a97-b771-dc48fbaa0fbb/resourceGroups/rg-lab-apic-central-3/providers/Microsoft.ApiCenter/services/apic-central-3/workspaces/default/apis/asana-prod?api-version=2024-06-01-preview" --body @_temp_api_asana-prod.json 
⚙️ Running: az apic api version create -g rg-lab-apic-central-3 -n apic-central-3 --api-id asana-prod --version-id 1-0-0 --title "1.0.0" --lifecycle-stage production 
⚙️ Running: az apic api definition create -g rg-lab-apic-central-3 -n apic-central-3 --api-id asana-prod --version-id 1-0-0 --definition-id default --title "Default" 
⚙️ Running: az apic api deployment list -g rg-lab-mcp-containers-prod-3 -n apic-fastmcp-prod-3-m7hknq27w253u --api-id asana -o json 
  ✅ asana     

---
## Part 6 — Unified Discovery from Central API Center (prod + staging)
---

> 💡 **Single pane of glass** — query one API Center, discover APIs from all environments.
> APIs are tagged and routed to their correct prod/staging environments.

In [29]:
# List everything in the central API Center
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed ALL APIs in Central API Center", "Failed")
if output.success:
    print(output.text)

output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} -o json', "", "")
if output.success and output.json_data:
    central_apis = output.json_data
    central_rest = sum(1 for api in central_apis if api.get('kind') == 'rest')
    central_mcp  = sum(1 for api in central_apis if api.get('kind') == 'mcp')
    central_a2a  = sum(1 for api in central_apis if api.get('kind') == 'a2a')

    # Count by stage (inferred from API name suffix)
    prod_count = sum(1 for api in central_apis if api['name'].endswith('-prod'))
    staging_count = sum(1 for api in central_apis if api['name'].endswith('-staging'))

    _rg2_total = sum(rg2_synced.values()) if 'rg2_synced' in globals() and isinstance(rg2_synced, dict) else 0
    _rg3_total = sum(rg3_synced.values()) if 'rg3_synced' in globals() and isinstance(rg3_synced, dict) else 0
    _rg4_total = sum(rg4_synced.values()) if 'rg4_synced' in globals() and isinstance(rg4_synced, dict) else 0
    sources = f"RG 2 ({_rg2_total}) + RG 3 ({_rg3_total})"
    if _rg4_total > 0:
        sources += f" + RG 4 ({_rg4_total})"
    print(f"\n📊 Central Catalog: {len(central_apis)} APIs — {central_rest} REST, {central_mcp} MCP, {central_a2a} A2A")
    print(f"   Environments:    {prod_count} prod, {staging_count} staging")
    print(f"   Aggregated from: {sources}")

⚙️ Running: az apic api list -g rg-lab-apic-central-3 -n apic-central-3 --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed ALL APIs in Central API Center ⌚ 13:45:28.109102 :5s]
Name                      Title                          Kind
------------------------  -----------------------------  ------
swagger-petstore          Swagger Petstore               rest
cloudflare-prod           Cloudflare (prod)              mcp
sentry-prod               Sentry (prod)                  mcp
paypal-prod               Paypal (prod)                  mcp
plaid-prod                Plaid (prod)                   mcp
asana-prod                Asana (prod)                   mcp
intercom-prod             Intercom (prod)                mcp
linear-prod               Linear (prod)                  mcp
atlassian-prod            Atlassian (prod)               mcp
square-prod               Square (prod)                  mcp
swagger-petstore-prod     Swagger Petstore (prod)        rest
weather

### 🔍 Filter: MCP servers with deployment endpoints from Central Catalog (prod + staging)

In [30]:
# Show MCP servers from central catalog with their deployment endpoints
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'mcp\'].{{Name:name, Title:title, Kind:kind}}" -o table',
    "MCP servers in Central Catalog", "Failed")
if output.success:
    print(output.text)

# Show deployments with runtime URIs for each MCP server, grouped by stage
print("\n🔗 MCP Server Endpoints (from Central Catalog deployments)")
print("=" * 80)
mcp_endpoints = {}
for api in central_apis:
    if api.get('kind') == 'mcp':
        stage = "prod" if api['name'].endswith('-prod') else "staging" if api['name'].endswith('-staging') else "?"
        deployments = get_apic_deployments(rg1_name, rg1_apic_name, api['name'])
        for dep in deployments:
            uris = dep.get('server', {}).get('runtimeUri', [])
            source = dep.get('title', '')
            for uri in uris:
                print(f"  [{stage:7}] {api['name']:30} → {uri}")
                mcp_endpoints[api['name']] = uri

print(f"\n📊 {len(mcp_endpoints)} MCP servers discoverable with invocation endpoints")

⚙️ Running: az apic api list -g rg-lab-apic-central-3 -n apic-central-3 --query "[?kind=='mcp'].{Name:name, Title:title, Kind:kind}" -o table 
✅ MCP servers in Central Catalog ⌚ 13:45:45.997403 :5s]
Name                    Title                          Kind
----------------------  -----------------------------  ------
cloudflare-prod         Cloudflare (prod)              mcp
sentry-prod             Sentry (prod)                  mcp
paypal-prod             Paypal (prod)                  mcp
plaid-prod              Plaid (prod)                   mcp
asana-prod              Asana (prod)                   mcp
intercom-prod           Intercom (prod)                mcp
linear-prod             Linear (prod)                  mcp
atlassian-prod          Atlassian (prod)               mcp
square-prod             Square (prod)                  mcp
weather-mcp-prod        Weather MCP (prod)             mcp
order-mcp-prod          Order Service MCP (prod)       mcp
catalog-mcp-prod        Produc

### 🔌 Dynamic connection — discover and invoke MCP servers from Central Catalog

An agent or client can:
1. Query the central API Center for `kind == 'mcp'`
2. Filter by environment (prod/staging) via deployment environment
3. Get the `runtimeUri` from deployments
4. Connect and invoke tools — **without knowing which APIM or RG hosts them**

In [ ]:
# Dynamically discover and invoke MCP servers from the central catalog
# Focus on PROD servers for the live demo invocation
print("🔌 Dynamic MCP Discovery + Invocation from Central Catalog (PROD)")
print("=" * 65)

# Step 1: Discover MCP servers
output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'mcp\']" -o json', "", "")
discovered_servers = output.json_data if output.success and output.json_data else []
prod_servers = [s for s in discovered_servers if s['name'].endswith('-prod')]
staging_servers = [s for s in discovered_servers if s['name'].endswith('-staging')]
print(f"\n  Step 1: Discovered {len(discovered_servers)} MCP servers ({len(prod_servers)} prod, {len(staging_servers)} staging)")

# Build gateway lookup from all envs
gw_lookup = {}  # gateway_url → api_key
for env in ENVS:
    e = envs[env]
    for key_prefix in ["rg2", "rg3"]:
        gw = e.get(f"{key_prefix}_gateway", e.get(f"{key_prefix}_gateway_url", "")).rstrip("/")
        api_key = e.get(f"{key_prefix}_api_key", "")
        if gw:
            gw_lookup[gw] = api_key

# Step 2: Get runtime URIs for prod servers (collect ALL deployment URIs, not just the first)
print(f"  Step 2: Resolving runtime endpoints for prod servers...")

invocable = {}
invocable_all = {}  # name → [all candidate URIs]
for api in prod_servers:
    deps = get_apic_deployments(rg1_name, rg1_apic_name, api['name'])
    all_uris = []
    for dep in deps:
        uris = dep.get('server', {}).get('runtimeUri', [])
        all_uris.extend(uris)
    if all_uris:
        seen = set()
        unique = [u for u in all_uris if u not in seen and not seen.add(u)]
        invocable_all[api['name']] = unique
        invocable[api['name']] = unique[0]

# Step 3: Invoke tools/list on APIM-hosted servers (with endpoint fallback)
our_servers = {}    # name → [(uri, headers), ...] all APIM-hosted candidates
our_headers = {}
external = {}
for k, uris in invocable_all.items():
    candidates = []
    ext_uri = None
    for v in uris:
        matched_gw = False
        for gw, api_key in gw_lookup.items():
            if v.startswith(gw):
                candidates.append((v, {"api-key": api_key}))
                matched_gw = True
                break
        if not matched_gw and not ext_uri:
            ext_uri = v
    if candidates:
        our_servers[k] = candidates
    elif ext_uri:
        external[k] = ext_uri

print(f"  Step 3: Invoking tools on {len(our_servers)} APIM-hosted prod servers "
      f"(skipping {len(external)} external 3rd-party servers)\n")

for name in sorted(our_servers.keys()):
    found = False
    for endpoint, hdrs in our_servers[name]:
        tools = list_mcp_tools(endpoint, label=name, extra_headers=hdrs)
        if tools:
            invocable[name] = endpoint      # update to working endpoint
            our_headers[name] = hdrs
            found = True
            break
    if not found:
        # Default to first candidate so cell 41 has something to try
        invocable[name] = our_servers[name][0][0]
        our_headers[name] = our_servers[name][0][1]
        print(f"  ⚠️  {name}: no tools returned (tried {len(our_servers[name])} endpoints)")

if external:
    print(f"\n  📋 External MCP servers (not invoked — require vendor auth):")
    for name, uri in sorted(external.items()):
        print(f"     {name:25} → {uri}")

print(f"\n📊 Discovered {len(invocable)} prod MCP servers, invoked {len(our_servers)} via APIM gateways")
print(f"   (+ {len(staging_servers)} staging MCP servers also registered in central catalog)")

🔌 Dynamic MCP Discovery + Invocation from Central Catalog (PROD)
⚙️ Running: az apic api list -g rg-lab-apic-central-3 -n apic-central-3 --query "[?kind=='mcp']" -o json 

  Step 1: Discovered 26 MCP servers (13 prod, 13 staging)
  Step 2: Resolving runtime endpoints for prod servers...
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-3 -n apic-central-3 --api-id cloudflare-prod -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-3 -n apic-central-3 --api-id sentry-prod -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-3 -n apic-central-3 --api-id paypal-prod -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-3 -n apic-central-3 --api-id plaid-prod -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-3 -n apic-central-3 --api-id asana-prod -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-3 -n apic-central-3 --api-id intercom-prod -o json 
⚙️ Running: az apic api deploy

---
## Part 7 — Agent Workflow: Cross-Environment Tool Chaining (prod)
---

> An agent discovers MCP servers from the **central catalog** (filtering to prod) and chains tools across **multiple APIM instances** — Weather from RG 2, Calculator from RG 3, etc.

### Simulated agent scenario:
1. **Discover** available MCP servers from central API Center (prod environment)
2. **Search catalog** → find a laptop
3. **Place order** → order it
4. **Calculate total** → compute with tax
5. **Check weather** → for delivery planning

In [ ]:
print("🤖 Agent Workflow — Cross-Environment Tool Chaining (PROD)")
print("=" * 65)

# Step 1: Agent discovers MCP servers from central catalog (prod only)
print("\n📋 Step 1: Agent queries central API Center for PROD MCP servers")
mcp_servers = {api['name']: api.get('title','') for api in prod_servers}
for name, title in sorted(mcp_servers.items()):
    endpoint = invocable.get(name, '—')
    # Determine which APIM hosts this endpoint
    hosted = "External"
    for gw in gw_lookup:
        if endpoint.startswith(gw):
            hosted = "APIM"
            break
    print(f"  • {title:30} [{hosted}] → {endpoint[:70]}...")

def get_headers_for(endpoint):
    """Return the correct APIM subscription header for an endpoint."""
    for gw, api_key in gw_lookup.items():
        if endpoint.startswith(gw):
            return {"api-key": api_key}
    return {}

def call_mcp_with_fallback(api_name, tool_name, arguments, label):
    """Call an MCP tool, trying all candidate endpoints from invocable_all on failure."""
    # Try the primary (best-known) endpoint first
    primary_ep = invocable.get(api_name)
    if primary_ep:
        result = call_mcp(primary_ep, tool_name, arguments, label, get_headers_for(primary_ep))
        if result is not None:
            return result

    # Fallback: try other candidate endpoints from invocable_all
    alt_uris = invocable_all.get(api_name, [])
    for uri in alt_uris:
        if uri == primary_ep:
            continue
        hdrs = get_headers_for(uri)
        if hdrs:  # Only try APIM-hosted endpoints
            result = call_mcp(uri, tool_name, arguments, f"{label} (alt)", hdrs)
            if result is not None:
                invocable[api_name] = uri  # remember working endpoint
                return result
    return None

# Step 2: Search catalog
print("\n🔍 Step 2: Agent calls catalog MCP → search for 'laptop'")
result = call_mcp_with_fallback('catalog-mcp-prod', "search_products", {"query": "laptop"}, "Catalog search")
if result:
    print(f"    → Found: {json.dumps(result)[:200]}")

# Step 3: Place order
print("\n📦 Step 3: Agent calls order MCP → place order")
result = call_mcp_with_fallback('order-mcp-prod', "place_order", {"product_id": "PROD-001", "quantity": 2}, "Place order")
if result:
    print(f"    → Order: {json.dumps(result)[:200]}")

# Step 4: Calculate total with tax
print("\n🧮 Step 4: Agent calls calculator MCP → compute total with tax")
result = call_mcp_with_fallback('calculator-mcp-prod', "calculate", {"operation": "multiply", "a": 999.99, "b": 1.08}, "Calculate total")
if result:
    total = result.get('result', result) if isinstance(result, dict) else result
    print(f"    → Total with 8% tax: ${total}")

# Step 5: Check weather for delivery
print("\n🌤️ Step 5: Agent calls weather MCP → check delivery weather")
result = call_mcp_with_fallback('weather-mcp-prod', "get_weather", {"city": "Seattle"}, "Weather check")
if result:
    print(f"    → Weather: {json.dumps(result)[:200]}")

print("\n" + "=" * 65)
print("✅ Agent workflow complete — 4 tools chained across PROD APIM gateways via central catalog!")

🤖 Agent Workflow — Cross-Environment Tool Chaining (PROD)

📋 Step 1: Agent queries central API Center for PROD MCP servers
  • Asana (prod)                   [External] → https://mcp.asana.com/sse...
  • Atlassian (prod)               [External] → https://mcp.atlassian.com/v1/sse...
  • Calculator MCP (prod)          [APIM] → https://apim-mcp-demo-prod-3.azure-api.net/calculator-mcp...
  • Product Catalog MCP (prod)     [APIM] → https://apim-mcp-demo-prod-3.azure-api.net/catalog-mcp...
  • Cloudflare (prod)              [External] → https://docs.mcp.cloudflare.com/sse...
  • Intercom (prod)                [External] → https://mcp.intercom.com/sse...
  • Linear (prod)                  [External] → https://mcp.linear.app/sse...
  • Order Service MCP (prod)       [APIM] → https://apim-mcp-demo-prod-3.azure-api.net/order-mcp...
  • Paypal (prod)                  [External] → https://mcp.paypal.com/sse...
  • Plaid (prod)                   [External] → https://api.dashboard.plaid.com/mcp/ss

---
## Part 8 — Deploy & Explore A2A Agents (RG 4) — prod + staging
---

> Deploy **A2A agents** (Agent-to-Agent protocol) alongside REST APIs for both environments.  
> Each agent exposes `/.well-known/agent.json` agent cards and communicates via the [A2A protocol](https://google.github.io/A2A/).
> 
> **Architecture**: REST API → APIM proxy (westeurope) → A2A agent (registered in APIC with `kind: a2a`)

> ⏱️ First deployment takes ~5-8 minutes per env (APIM provisioning). Subsequent runs are incremental.

In [ ]:
# Deploy RG 4 (A2A agents) for each environment
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

for env in ENVS:
    e = envs[env]
    print(f"\n{'='*60}")
    print(f"  Deploying RG 4 — {env.upper()}: {e['rg4_name']}")
    print(f"{'='*60}")

    utils.create_resource_group(e["rg4_name"], rg_location)

    bicep_parameters = {
        "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
        "contentVersion": "1.0.0.0",
        "parameters": {
            "apimSku": { "value": apim_sku },
            "apimName": { "value": e["rg4_apim_name"] },
            "apimLocation": { "value": apim_location },
            "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
            "apicLocation": { "value": rg_location },
            "apicServiceNamePrefix": { "value": e["rg4_apic_prefix"] }
        }
    }

    params_file = f'params-a2a-initial-{env}.json'
    with open(params_file, 'w') as f:
        f.write(json.dumps(bicep_parameters))

    output = utils.run(
        f"az deployment group create --name {e['rg4_deployment_name']} --resource-group {e['rg4_name']} "
        f"--template-file demo-a2a-initial.bicep --parameters {params_file}",
        f"✅ RG 4 [{env}] deployment succeeded (Title + Outline agents)",
        f"❌ RG 4 [{env}] deployment failed"
    )

    # Retrieve outputs
    output = utils.run(f"az deployment group show --name {e['rg4_deployment_name']} -g {e['rg4_name']}", "", "")

    has_outputs = False
    if output.success and output.json_data:
        outputs = output.json_data.get('properties', {}).get('outputs')
        if outputs:
            has_outputs = True

    if has_outputs:
        e["rg4_apim"]   = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service')
        e["rg4_apim_gateway"] = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway')
        e["rg4_apic"]   = utils.get_deployment_output(output, 'apicServiceName', 'API Center')
        e["rg4_apic_api_env"] = utils.get_deployment_output(output, 'apicApiEnvironmentName', 'APIC API Env')
        e["rg4_apic_a2a_env"] = utils.get_deployment_output(output, 'apicA2aEnvironmentName', 'APIC A2A Env')
        rg4_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
        e["rg4_api_key"] = rg4_subscriptions[0].get("key")
    else:
        utils.print_info("Deployment outputs unavailable — resolving from live resources")
        e["rg4_apim"] = e["rg4_apim_name"]
        o = utils.run(f'az apim show --name {e["rg4_apim_name"]} -g {e["rg4_name"]} --query gatewayUrl -o tsv', "APIM Gateway", "")
        e["rg4_apim_gateway"] = o.text.strip() if o.success else ""
        o = utils.run(f'az apic list -g {e["rg4_name"]} --query "[0].name" -o tsv', "API Center", "")
        e["rg4_apic"] = o.text.strip() if o.success else ""
        e["rg4_apic_api_env"] = "api"
        e["rg4_apic_a2a_env"] = "a2a"
        sub_id = utils.get_current_subscription()
        o = utils.run(
            f'az rest --method POST --url "https://management.azure.com/subscriptions/{sub_id}'
            f'/resourceGroups/{e["rg4_name"]}/providers/Microsoft.ApiManagement/service/{e["rg4_apim_name"]}'
            f'/subscriptions/subscription1/listSecrets?api-version=2022-08-01" --query primaryKey -o tsv',
            "API Key", "")
        e["rg4_api_key"] = o.text.strip() if o.success else ""

    print(f"\n📋 RG 4 [{env}] Resources:")
    print(f"  APIM:     {e.get('rg4_apim','?')}")
    print(f"  Gateway:  {e.get('rg4_apim_gateway','?')}")
    print(f"  APIC:     {e.get('rg4_apic','?')}")
    print(f"  API Key:  ****{e.get('rg4_api_key','')[-4:] if e.get('rg4_api_key') else '????'}")

    # Deploy Summary agent add-on
    summary_params = {
        "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
        "contentVersion": "1.0.0.0",
        "parameters": {
            "apimServiceName": { "value": e["rg4_apim"] },
            "apicServiceName": { "value": e["rg4_apic"] },
            "apicApiEnvironmentName": { "value": e["rg4_apic_api_env"] },
            "apicA2aEnvironmentName": { "value": e["rg4_apic_a2a_env"] }
        }
    }

    params_file = f'params-a2a-summary-{env}.json'
    with open(params_file, 'w') as f:
        f.write(json.dumps(summary_params))

    output = utils.run(
        f"az deployment group create --name {e['rg4_summary_deployment_name']} --resource-group {e['rg4_name']} "
        f"--template-file demo-a2a-add-summary.bicep --parameters {params_file}",
        f"✅ Summary agent add-on deployed in RG 4 [{env}]",
        f"❌ Summary agent deployment failed [{env}]"
    )

⚙️ Running: az group show --name rg-lab-a2a-demo-2 
👉🏽 Resource group rg-lab-a2a-demo-2 does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name rg-lab-a2a-demo-2 --location uksouth --tags source=ai-gateway 
✅ Resource group 'rg-lab-a2a-demo-2' created ⌚ 21:09:32.008745 :6s]
⚙️ Running: az deployment group create --name a2a-initial --resource-group rg-lab-a2a-demo-2 --template-file demo-a2a-initial.bicep --parameters params-a2a-initial.json 
✅ ✅ RG 4 deployment 'a2a-initial' succeeded (Title + Outline agents) ⌚ 21:12:49.239397 :17s]
⚙️ Running: az deployment group show --name a2a-initial -g rg-lab-a2a-demo-2 
👉🏽 APIM Service: apim-a2a-2
👉🏽 APIM Gateway: https://apim-a2a-2.azure-api.net
👉🏽 API Center: apic-a2a-2-pj4tltq7j77t4
👉🏽 APIC API Env: api
👉🏽 APIC A2A Env: a2a

📋 RG 4 Resources:
  APIM:     apim-a2a-2
  Gateway:  https://apim-a2a-2.azure-api.net
  APIC:     apic-a2a-2-pj4tltq7j77t4
  API Key:  ****beae
⚙️ Running: az deployment group create --name

In [ ]:
# List APIs in the A2A API Center — shows all protocol types — prod + staging
for env in ENVS:
    e = envs[env]
    print(f"\n{'='*70}")
    print(f"  📋 APIs in RG 4 [{env.upper()}] API Center: {e.get('rg4_apic','?')}")
    print(f"{'='*70}")

    output = utils.run(
        f'az apic api list -g {e["rg4_name"]} -n {e.get("rg4_apic","")} '
        f'--query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
        f"Listed APIs in A2A [{env}] API Center", "Failed")
    if output.success:
        print(output.text)

    output = utils.run(f'az apic api list -g {e["rg4_name"]} -n {e.get("rg4_apic","")} -o json', "", "")
    if output.success and output.json_data:
        e["rg4_apis"] = output.json_data
        rg4_rest = sum(1 for api in e["rg4_apis"] if api.get('kind') == 'rest')
        rg4_mcp  = sum(1 for api in e["rg4_apis"] if api.get('kind') == 'mcp')
        rg4_a2a  = sum(1 for api in e["rg4_apis"] if api.get('kind') == 'a2a')
        print(f"\n📊 RG 4 [{env}]: {len(e['rg4_apis'])} APIs — {rg4_rest} REST, {rg4_mcp} MCP, {rg4_a2a} A2A agents")

📋 APIs registered in RG 4 API Center
⚙️ Running: az apic api list -g rg-lab-a2a-demo-2 -n apic-a2a-2-pj4tltq7j77t4 --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in A2A API Center ⌚ 21:14:08.902033 :5s]
Name                   Title                        Kind
---------------------  ---------------------------  ------
intercom               Intercom                     mcp
cloudflare             Cloudflare                   mcp
sentry                 Sentry                       mcp
atlassian              Atlassian                    mcp
paypal                 Paypal                       mcp
plaid                  Plaid                        mcp
asana                  Asana                        mcp
linear                 Linear                       mcp
square                 Square                       mcp
swagger-petstore       Swagger Petstore             rest
title-generator-api    Title Generator API          rest
outline-generator-api  Outline Genera

### 🔍 Explore A2A agents — APIM APIs and agent cards (prod + staging)

A2A agents are exposed through APIM just like MCP servers. Each agent serves:
- `/.well-known/agent.json` — **Agent Card** (name, description, skills, capabilities)
- `/` — A2A JSON-RPC endpoint for task submission

In [ ]:
# List APIM APIs in RG 4 and fetch agent cards — prod + staging
subscription_id = utils.get_current_subscription()

for env in ENVS:
    e = envs[env]
    print(f"\n{'='*70}")
    print(f"  📋 APIM APIs in RG 4 [{env.upper()}] — {e.get('rg4_apim','?')}")
    print(f"{'='*70}")

    token = subprocess.run(["az", "account", "get-access-token", "--query", "accessToken", "-o", "tsv"],
                           capture_output=True, text=True, shell=True).stdout.strip()
    apim_url = (f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{e['rg4_name']}"
                f"/providers/Microsoft.ApiManagement/service/{e.get('rg4_apim','')}/apis?api-version=2024-06-01-preview")
    resp = requests.get(apim_url, headers={"Authorization": f"Bearer {token}"})
    if resp.status_code == 200:
        apis = resp.json().get('value', [])
        for api in apis:
            name = api['name']
            display = api['properties'].get('displayName', '')
            path = api['properties'].get('path', '')
            api_type = api['properties'].get('type', 'http')
            print(f"  {name:30} path=/{path:20} type={api_type:8} {display}")
        print(f"\n📊 {len(apis)} APIs in APIM ({e.get('rg4_apim','')})")

    # Fetch agent cards from A2A agents
    rg4_apis = e.get("rg4_apis", [])
    a2a_agents = [api for api in rg4_apis if api.get('kind') == 'a2a']

    print(f"\n🃏 Fetching A2A Agent Cards [{env}] via APIM")
    print("-" * 70)

    for agent in a2a_agents:
        agent_name = agent['name']
        deps = get_apic_deployments(e["rg4_name"], e.get("rg4_apic", ""), agent_name)
        for dep in deps:
            for uri in dep.get('server', {}).get('runtimeUri', []):
                card_url = f"{uri}/.well-known/agent.json"
                try:
                    card_resp = requests.get(card_url, timeout=10)
                    if card_resp.status_code == 200:
                        card = card_resp.json()
                        print(f"\n  🤖 {card.get('name', agent_name)} [{env}]")
                        print(f"     Description: {card.get('description', 'N/A')}")
                        print(f"     Version: {card.get('version', 'N/A')}")
                        skills = card.get('skills', [])
                        for skill in skills:
                            print(f"     Skill: {skill.get('name', '')} — {skill.get('description', '')}")
                        caps = card.get('capabilities', {})
                        print(f"     Streaming: {caps.get('streaming', False)}")
                    else:
                        print(f"\n  ⚠️ {agent_name} [{env}]: agent card returned {card_resp.status_code}")
                except Exception as ex:
                    print(f"\n  ⚠️ {agent_name} [{env}]: {ex}")
                break

📋 APIM APIs in RG 4
  outline-agent                  path=/outline-agent        type=http     Outline Generator A2A Agent
  outline-generator-api          path=/outline-generator    type=http     Outline Generator API
  summary-agent                  path=/summary-agent        type=http     Summary Generator A2A Agent
  summary-generator-api          path=/summary-generator    type=http     Summary Generator API
  title-agent                    path=/title-agent          type=http     Title Generator A2A Agent
  title-generator-api            path=/title-generator      type=http     Title Generator API

📊 6 APIs in APIM (apim-a2a-2)

🃏 Fetching A2A Agent Cards via APIM
----------------------------------------------------------------------
⚙️ Running: az apic api deployment list -g rg-lab-a2a-demo-2 -n apic-a2a-2-pj4tltq7j77t4 --api-id title-agent -o json 

  🤖 Title Generator Agent
     Description: An intelligent title generator agent. I can help you generate catchy, SEO-friendly titl

---
## Part 9 — Aggregate A2A Agents to Central Catalog (prod → a2a-prod, staging → a2a-staging)
---

> Sync APIs from **RG 4** (A2A agents + REST + MCP) into the **Central API Center**.  
> Uses the same `register_api_in_central()` function — it routes each API to the correct prod/staging environment automatically.

In [ ]:
# Aggregate from RG 4 (A2A agents + REST + MCP) — prod + staging
rg4_synced = {}
for env in ENVS:
    e = envs[env]
    apis = get_apic_apis(e["rg4_name"], e.get("rg4_apic", ""))
    e["rg4_apis"] = apis
    print(f"\n📥 Syncing {len(apis)} APIs from RG 4 [{env}] ({e.get('rg4_apic','')}) → Central APIC ({env} envs)")
    print("=" * 70)

    synced = 0
    for api in apis:
        uris = register_api_in_central(api, e["rg4_name"], e.get("rg4_apic", ""), f"rg4-{env}", env)
        uri_str = uris[0] if uris else "—"
        kind = api.get('kind', '?')
        icon = {'a2a': '🤖', 'mcp': '🔌', 'rest': '🌐'}.get(kind, '📦')
        print(f"  {icon} {api['name']:25} ({kind:4}) → {env}  {uri_str}")
        synced += 1

    rg4_rest = sum(1 for api in apis if api.get('kind') == 'rest')
    rg4_mcp  = sum(1 for api in apis if api.get('kind') == 'mcp')
    rg4_a2a  = sum(1 for api in apis if api.get('kind') == 'a2a')
    rg4_synced[env] = synced
    print(f"\n📊 Synced {synced} APIs from RG 4 [{env}] ({rg4_rest} REST, {rg4_mcp} MCP, {rg4_a2a} A2A)")

⚙️ Running: az apic api list -g rg-lab-a2a-demo-2 -n apic-a2a-2-pj4tltq7j77t4 -o json 
📥 Syncing 16 APIs from RG 4 (apic-a2a-2-pj4tltq7j77t4) → Central APIC
⚙️ Running: az rest --method PUT --url "https://management.azure.com/subscriptions/31613fe0-1e9b-4a97-b771-dc48fbaa0fbb/resourceGroups/rg-lab-apic-central-2/providers/Microsoft.ApiCenter/services/apic-central-2/workspaces/default/apis/intercom?api-version=2024-06-01-preview" --body @_temp_api_intercom.json 
⚙️ Running: az apic api version create -g rg-lab-apic-central-2 -n apic-central-2 --api-id intercom --version-id 1-0-0 --title "1.0.0" --lifecycle-stage production 
⚙️ Running: az apic api definition create -g rg-lab-apic-central-2 -n apic-central-2 --api-id intercom --version-id 1-0-0 --definition-id default --title "Default" 
⚙️ Running: az apic api deployment list -g rg-lab-a2a-demo-2 -n apic-a2a-2-pj4tltq7j77t4 --api-id intercom -o json 
  🔌 intercom                  (mcp ) → https://mcp.intercom.com/sse
⚙️ Running: az rest 

---
## Part 10 — Unified Governance: REST + MCP + A2A in One Catalog (prod + staging)
---

> 💡 The **Central API Center** now contains APIs from **all 3 protocol types** across distributed resource groups, with separate **prod** and **staging** environments.

### What's in the Central Catalog
| Protocol | Discovery | Environment |
|----------|-----------|-------------|
| **REST** | `kind: rest` → `api-prod` / `api-staging` | Per-stage |
| **MCP** | `kind: mcp` → `mcp-prod` / `mcp-staging` | Per-stage |
| **A2A** | `kind: a2a` → `a2a-prod` / `a2a-staging` | Per-stage |

In [ ]:
# ── 10a. Full catalog view — all protocols, all environments ─────────────────

# Refresh the central catalog
output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} -o json', "", "")
central_apis = output.json_data if output.success and output.json_data else []

# Categorize by kind
central_rest = [a for a in central_apis if a.get("kind") == "rest"]
central_mcp  = [a for a in central_apis if a.get("kind") == "mcp"]
central_a2a  = [a for a in central_apis if a.get("kind") == "a2a"]

# Count by stage
prod_apis    = [a for a in central_apis if a['name'].endswith('-prod')]
staging_apis = [a for a in central_apis if a['name'].endswith('-staging')]

print(f"╔══════════════════════════════════════════════════════╗")
print(f"║   Central API Catalog — Unified Discovery Hub       ║")
print(f"╠══════════════════════════════════════════════════════╣")
print(f"║  Total APIs registered:  {len(central_apis):<26} ║")
print(f"║  ─────────────────────────────────────────────────── ║")
print(f"║  🌐 REST APIs:           {len(central_rest):<26} ║")
print(f"║  🔌 MCP Servers:         {len(central_mcp):<26} ║")
print(f"║  🤖 A2A Agents:          {len(central_a2a):<26} ║")
print(f"║  ─────────────────────────────────────────────────── ║")
print(f"║  🟢 Production:          {len(prod_apis):<26} ║")
print(f"║  🟡 Staging:             {len(staging_apis):<26} ║")
print(f"╚══════════════════════════════════════════════════════╝")

# Print each API with its protocol badge and stage
print(f"\n{'Stage':<9} {'Kind':<6} {'API Name':<45} {'Title'}")
print("─" * 90)
icons = {"rest": "🌐", "mcp": "🔌", "a2a": "🤖"}
for api in sorted(central_apis, key=lambda a: (a['name'].split('-')[-1], a.get("kind", ""), a.get("name", ""))):
    kind = api.get("kind", "?")
    icon = icons.get(kind, "❓")
    stage = "prod" if api['name'].endswith('-prod') else "staging" if api['name'].endswith('-staging') else "?"
    stage_icon = "🟢" if stage == "prod" else "🟡"
    print(f"{stage_icon} {stage:<7} {icon} {kind:<4} {api['name']:<45} {api.get('title', '')}")

⚙️ Running: az apic api list -g rg-lab-apic-central-2 -n apic-central-2 -o json 
╔══════════════════════════════════════════════════════╗
║   Central API Catalog — Unified Discovery Hub       ║
╠══════════════════════════════════════════════════════╣
║  Total APIs registered:  24                         ║
║  ─────────────────────────────────────────────────── ║
║  🌐 REST APIs:           8                          ║
║  🔌 MCP Servers:         13                         ║
║  🤖 A2A Agents:          3                          ║
╚══════════════════════════════════════════════════════╝

Kind   API Name                                 Title
────────────────────────────────────────────────────────────────────────────────
🤖 a2a  outline-agent                            Outline Generator A2A Agent
🤖 a2a  summary-agent                            Summary Generator A2A Agent
🤖 a2a  title-agent                              Title Generator A2A Agent
🔌 mcp  asana                                    Asan

### 10b. Dynamic A2A Agent Discovery & Invocation (prod + staging)

> An **AI agent** can discover A2A agents from the central catalog, filtered by environment:  
> 1. Query API Center for APIs with `kind == "a2a"` (optionally filter by `-prod` or `-staging` suffix)
> 2. Get the deployment `runtimeUri` for each agent (routed via prod/staging environments)
> 3. Fetch the **Agent Card** at `{runtimeUri}/.well-known/agent.json`  
> 4. Invoke the agent via A2A protocol (`tasks/send`)  
>
> All traffic flows through **APIM** (westeurope) — giving you rate limiting, auth, monitoring, and governance for free.

In [ ]:
# ── 10b. Dynamic A2A agent discovery from central catalog (prod + staging) ────

# Step 1: Query central APIC for A2A agents
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'a2a\']" -o json', "", "")
a2a_agents_central = output.json_data if output.success and output.json_data else []
prod_a2a = [a for a in a2a_agents_central if a['name'].endswith('-prod')]
staging_a2a = [a for a in a2a_agents_central if a['name'].endswith('-staging')]
print(f"Found {len(a2a_agents_central)} A2A agents — {len(prod_a2a)} prod, {len(staging_a2a)} staging\n")

# Step 2: For each agent, get its deployment runtimeUri
discovered_agents = []

for agent in a2a_agents_central:
    api_name = agent["name"]
    stage = "prod" if api_name.endswith('-prod') else "staging"
    deployments = get_apic_deployments(rg1_name, rg1_apic_name, api_name)

    for dep in deployments:
        runtime_uri = dep.get("server", {}).get("runtimeUri", [None])[0]
        env_id = dep.get("environmentId", "")
        env_name = env_id.split("/")[-1] if env_id else ""

        if runtime_uri and env_name.startswith("a2a"):
            discovered_agents.append({
                "name": api_name,
                "title": agent.get("title", api_name),
                "runtime_uri": runtime_uri,
                "stage": stage
            })
            stage_icon = "🟢" if stage == "prod" else "🟡"
            print(f"{stage_icon} {api_name} [{stage}]")
            print(f"   Runtime: {runtime_uri}")

# Step 3: Fetch Agent Cards (via APIM gateway)
print(f"\n{'═' * 70}")
print(f"Agent Cards (fetched from .well-known/agent.json via APIM)")
print(f"{'═' * 70}")

for agent in discovered_agents:
    card_url = f"{agent['runtime_uri']}/.well-known/agent.json"
    try:
        resp = requests.get(card_url, timeout=10)
        if resp.status_code == 200:
            card = resp.json()
            stage_icon = "🟢" if agent['stage'] == "prod" else "🟡"
            print(f"\n{stage_icon} 🤖 {card.get('name', agent['name'])} [{agent['stage']}]")
            print(f"   Description: {card.get('description', 'N/A')}")
            print(f"   Version:     {card.get('version', 'N/A')}")
            skills = card.get("skills", [])
            if skills:
                for skill in skills:
                    print(f"   Skill:       {skill.get('name', '?')} — {skill.get('description', '')}")
            caps = card.get("capabilities", {})
            if caps:
                print(f"   Capabilities: streaming={caps.get('streaming', False)}, pushNotifications={caps.get('pushNotifications', False)}")
        else:
            print(f"\n⚠️  {agent['name']}: HTTP {resp.status_code}")
    except Exception as ex:
        print(f"\n⚠️  {agent['name']}: {ex}")

⚙️ Running: az apic api list -g rg-lab-apic-central-2 -n apic-central-2 --query "[?kind=='a2a']" -o json 
Found 3 A2A agents in the central catalog

⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id outline-agent -o json 
🤖 outline-agent
   Runtime: https://apim-a2a-2.azure-api.net/outline-agent
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id title-agent -o json 
🤖 title-agent
   Runtime: https://apim-a2a-2.azure-api.net/title-agent
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id summary-agent -o json 
🤖 summary-agent
   Runtime: https://apim-a2a-2.azure-api.net/summary-agent

══════════════════════════════════════════════════════════════════════
Agent Cards (fetched from .well-known/agent.json via APIM)
══════════════════════════════════════════════════════════════════════

🤖 Outline Generator Agent
   Description: An intelligent outline generator agent. I 

### 10c. Governance Through APIM — All Protocols, All Environments, One Gateway

> Every API call — **REST**, **MCP**, or **A2A** — flows through **Azure API Management** (westeurope).  
> APIM provides **consistent governance** regardless of the protocol or environment (prod/staging):  
>
> | Governance Feature | REST | MCP | A2A |
> |---|---|---|---|
> | **Authentication** | ✅ subscription-key / OAuth2 | ✅ subscription-key / OAuth2 | ✅ subscription-key / OAuth2 |
> | **Rate Limiting** | ✅ per-subscription quotas | ✅ per-subscription quotas | ✅ per-subscription quotas |
> | **Request Logging** | ✅ App Insights | ✅ App Insights | ✅ App Insights |
> | **Token Metrics** | ✅ via emit-token-metric | ✅ via emit-token-metric | ✅ via emit-token-metric |
> | **Circuit Breaking** | ✅ backend pool | ✅ backend pool | ✅ backend pool |
> | **IP Filtering** | ✅ policy | ✅ policy | ✅ policy |
>
> The cell below queries **APIM analytics** across all 6 APIM instances (prod + staging × RG 2/3/4).

In [ ]:
# ── 10c. Governance metrics — APIM analytics across all protocols + environments ──

from datetime import datetime, timedelta, timezone

end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days=7)
time_filter = f"timestamp ge datetime'{start_time.strftime('%Y-%m-%dT%H:%M:%SZ')}' and timestamp le datetime'{end_time.strftime('%Y-%m-%dT%H:%M:%SZ')}'"
timespan = f"{start_time.strftime('%Y-%m-%dT%H:%M:%SZ')}/{end_time.strftime('%Y-%m-%dT%H:%M:%SZ')}"

access_token = subprocess.run(
    ["az", "account", "get-access-token", "--query", "accessToken", "-o", "tsv"],
    capture_output=True, text=True, shell=True).stdout.strip()
headers = {"Authorization": f"Bearer {access_token}"}

# Build APIM instance list from all envs
apim_instances = []
for env in ENVS:
    e = envs[env]
    apim_instances.append((e["rg2_name"], e.get("rg2_apim", e["rg2_apim_name"]), f"RG 2 — REST→MCP ({env})"))
    apim_instances.append((e["rg3_name"], e.get("rg3_apim_service", e["rg3_apim_name"]), f"RG 3 — FastMCP ({env})"))
    apim_instances.append((e["rg4_name"], e.get("rg4_apim", e["rg4_apim_name"]), f"RG 4 — A2A ({env})"))

for rg, apim, label in apim_instances:
    resource_uri = (
        f"/subscriptions/{subscription_id}/resourceGroups/{rg}"
        f"/providers/Microsoft.ApiManagement/service/{apim}"
    )

    # ── Try reports/byApi first (StandardV2+) ──
    report_url = f"https://management.azure.com{resource_uri}/reports/byApi"
    resp = requests.get(report_url, headers=headers, params={
        "$filter": time_filter, "api-version": "2023-09-01-preview"
    }, timeout=30)

    print(f"╔══════════════════════════════════════════════════════════════════════╗")
    print(f"║   {label:<65} ║")
    print(f"╠══════════════════════════════════════════════════════════════════════╣")

    if resp.status_code == 200:
        reports = resp.json().get("value", [])
        if reports:
            print(f"║  {'API Name':<30} {'Calls':>8} {'Success':>8} {'Errors':>8} {'Avg ms':>8} ║")
            print(f"║  {'─'*30} {'─'*8} {'─'*8} {'─'*8} {'─'*8} ║")
            total_calls = 0
            for r in sorted(reports, key=lambda x: x.get("callCountTotal", 0), reverse=True):
                name = r.get("name", "?")[:30]
                calls = r.get("callCountTotal", 0)
                success = r.get("callCountSuccess", 0)
                errors = r.get("callCountFailed", 0) + r.get("callCountOther", 0)
                avg_time = r.get("apiTimeAvg", 0)
                total_calls += calls
                print(f"║  {name:<30} {calls:>8} {success:>8} {errors:>8} {avg_time:>7.0f} ║")
            print(f"║  {'─'*30} {'─'*8} {'─'*8} {'─'*8} {'─'*8} ║")
            print(f"║  {'TOTAL':<30} {total_calls:>8}                          ║")
        else:
            print(f"║  No traffic recorded in the last 7 days                          ║")
    else:
        metrics_url = f"https://management.azure.com{resource_uri}/providers/microsoft.insights/metrics"
        mresp = requests.get(metrics_url, headers=headers, params={
            "api-version": "2023-10-01",
            "metricnames": "Requests,SuccessfulRequests,UnauthorizedRequests,FailedRequests",
            "timespan": timespan, "interval": "P7D", "aggregation": "Total",
        }, timeout=30)
        if mresp.status_code == 200:
            metric_data = {}
            for m in mresp.json().get("value", []):
                mname = m.get("name", {}).get("value", "?")
                total = sum(
                    int(dp.get("total", 0))
                    for ts in m.get("timeseries", [])
                    for dp in ts.get("data", [])
                    if dp.get("total") is not None
                )
                metric_data[mname] = total
            print(f"║  (Azure Monitor — per-API breakdown requires StandardV2+)        ║")
            print(f"║  {'─'*30} {'─'*8} {'─'*8} {'─'*8} {'─'*8} ║")
            print(f"║  {'Total Requests':<30} {metric_data.get('Requests', 0):>8}                          ║")
            print(f"║  {'Successful':<30} {metric_data.get('SuccessfulRequests', 0):>8}                          ║")
            print(f"║  {'Unauthorized':<30} {metric_data.get('UnauthorizedRequests', 0):>8}                          ║")
            print(f"║  {'Failed':<30} {metric_data.get('FailedRequests', 0):>8}                          ║")
        else:
            print(f"║  ⚠️  Could not fetch metrics (HTTP {mresp.status_code})              ║")

    print(f"╚══════════════════════════════════════════════════════════════════════╝\n")

print(f"✅ APIM provides consistent governance across REST, MCP, and A2A — in both prod and staging")
print(f"   — subscription keys, rate limits, logging, and analytics — all from {len(apim_instances)} APIM gateways in {apim_location}.")

╔══════════════════════════════════════════════════════════════════════╗
║   RG 2 — REST→MCP — apim-mcp-demo-2                                 ║
╠══════════════════════════════════════════════════════════════════════╣
║  ⚠️  Could not fetch metrics (HTTP 400)              ║
╚══════════════════════════════════════════════════════════════════════╝

╔══════════════════════════════════════════════════════════════════════╗
║   RG 3 — FastMCP — apim-fastmcp-2                                  ║
╠══════════════════════════════════════════════════════════════════════╣
║  ⚠️  Could not fetch metrics (HTTP 400)              ║
╚══════════════════════════════════════════════════════════════════════╝

╔══════════════════════════════════════════════════════════════════════╗
║   RG 4 — A2A Agents — apim-a2a-2                                      ║
╠══════════════════════════════════════════════════════════════════════╣
║  ⚠️  Could not fetch metrics (HTTP 400)              ║
╚═════════════════════════

---
## ✅ Demo Summary
---

In [ ]:
print("=" * 70)
print("  🎬  FEDERATED API GOVERNANCE DEMO — SUMMARY")
print("=" * 70)
print()
print("  Architecture (prod + staging, deployed by this notebook):")
print(f"    Region Layout:  APIM in {apim_location}, RGs in {rg_location}")
print(f"    Central APIC:   {rg1_apic_name} ({rg1_location})")
for env in ENVS:
    e = envs[env]
    stage_icon = "🟢" if env == "prod" else "🟡"
    print(f"    {stage_icon} {env.upper()}:")
    print(f"      RG 2 — REST→MCP:   {e.get('rg2_apim', e['rg2_apim_name'])} ({e['rg2_name']})")
    print(f"      RG 3 — FastMCP:    {e.get('rg3_apic', '?')} ({e['rg3_name']})")
    print(f"      RG 4 — A2A Agents: {e.get('rg4_apic', '?')} ({e['rg4_name']})")
print()
print("  API Inventory:")
for env in ENVS:
    e = envs[env]
    _rg2 = e.get("rg2_apis", [])
    _rg3 = e.get("rg3_apis", [])
    _rg4 = e.get("rg4_apis", [])
    _rg2_rest = sum(1 for a in _rg2 if a.get('kind') == 'rest')
    _rg2_mcp  = sum(1 for a in _rg2 if a.get('kind') == 'mcp')
    _rg3_mcp  = sum(1 for a in _rg3 if a.get('kind') == 'mcp')
    _rg4_rest = sum(1 for a in _rg4 if a.get('kind') == 'rest')
    _rg4_a2a  = sum(1 for a in _rg4 if a.get('kind') == 'a2a')
    stage_icon = "🟢" if env == "prod" else "🟡"
    print(f"    {stage_icon} {env.upper()}: RG 2={len(_rg2)} ({_rg2_rest}R/{_rg2_mcp}M) | RG 3={len(_rg3)} ({_rg3_mcp}M) | RG 4={len(_rg4)} ({_rg4_rest}R/{_rg4_a2a}A)")

_c_rest = len(central_rest) if isinstance(central_rest, list) else 0
_c_mcp  = len(central_mcp)  if isinstance(central_mcp, list)  else 0
_c_a2a  = len(central_a2a)  if isinstance(central_a2a, list)  else 0
_c_prod = len(prod_apis)    if isinstance(prod_apis, list)    else 0
_c_stg  = len(staging_apis) if isinstance(staging_apis, list) else 0
print(f"    Central: {len(central_apis)} APIs ({_c_rest}R, {_c_mcp}M, {_c_a2a}A) — {_c_prod} prod, {_c_stg} staging")
print()
print("  Key Takeaways:")
print("    • Central API Center = single pane of glass for REST + MCP + A2A discovery")
print("    • Prod and staging APIs routed to separate APIC environments automatically")
print("    • APIM (westeurope) = invocation layer with subscription keys, rate limits, and logging")
print("    • StandardV2 APIM = built-in analytics reports for per-API usage across all protocols")
print("    • A2A agents discoverable alongside MCP servers and REST APIs")
print("    • Agents discover from Central APIC, invoke through distributed prod/staging APIMs")
print("    • New APIs auto-registered in local APIC, synced to central on demand")
print("    • Works across subscriptions, resource groups, regions, and environments")
print("=" * 70)

  🎬  FEDERATED API GOVERNANCE DEMO — SUMMARY

  Architecture (all deployed by this notebook):
    RG 2 — REST→MCP:      apim-mcp-demo-2 (rg-lab-mcp-demo-2)
    RG 3 — FastMCP:        apic-fastmcp-2-brasrj7hswmuc (rg-lab-mcp-containers-2)
    RG 4 — A2A Agents:     apic-a2a-2-pj4tltq7j77t4 (rg-lab-a2a-demo-2)
    RG 1 — Central APIC:   apic-central-2 (swedencentral)

  API Inventory:
    RG 2: 18 APIs (5 REST, 13 MCP)
    RG 3: 14 APIs (13 MCP)
    RG 4: 16 APIs (4 REST, 9 MCP, 3 A2A)
    Central: 24 APIs (8 REST, 13 MCP, 3 A2A)

  Key Takeaways:
    • Central API Center = single pane of glass for REST + MCP + A2A discovery
    • APIM = invocation layer with subscription keys, rate limits, and logging
    • A2A agents are discoverable alongside MCP servers and REST APIs
    • Agent cards provide self-describing metadata for A2A agents
    • Agents discover from Central APIC, invoke through distributed APIMs
    • New APIs auto-registered in local APIC, synced to central on demand
    • 

---
## 🗑️ Clean up resources

Uncomment and run to delete **all 7 resource groups** (prod + staging × 3 + central) created by this demo.  
Or use [clean-up-resources.ipynb](clean-up-resources.ipynb).

In [ ]:
# Uncomment to delete all demo resources (uses current IDX value):

# all_rgs = [rg1_name]
# for env in ENVS:
#     e = envs[env]
#     all_rgs.extend([e["rg2_name"], e["rg3_name"], e["rg4_name"]])
# for rg in all_rgs:
#     utils.run(f"az group delete --name {rg} --yes --no-wait",
#               f"✅ {rg} deletion initiated", f"❌ Failed to delete {rg}")
# print(f"💡 To redeploy, increment IDX to {IDX + 1} in the variables cell")

print("⚠️  Uncomment the lines above to delete all 7 resource groups (prod + staging + central)")